# Orizon: Correlações entre Exames

Vamos entender como os serviços se relacionam entre si computando a frequencia relativa à cada um deles por ID conta. Por exemplo, para um serviço X, selecionamos todos os id_contas em que ele aparece, e então computamos a frequencia em que os outros serivços aparecem junto deste serviço. 

In [49]:
import pandas as pd
import numpy as np
from IPython.display import display

Lendo os dados para a dataframe:

In [50]:
df = pd.read_csv("../orizon/full_extracts/data_30726034.csv", encoding = "ISO-8859-1")
master = "'30726034'"

In [51]:
df.head()

,'categoria','id_cliente','cliente','id_lote','id_conta','guia_prestador','id_item','id_prestador','prestador','cnpj',...,'valor','origem','cnpj_cpf_codnaoperadora_executante','data','ano_mes',Unnamed: 47,Unnamed: 48,Unnamed: 49,Unnamed: 50,Unnamed: 51
0,'Seguradora Especializada em Saude','48','BRADESCO SAÃDE','42704626','1135485394','3280780','486879191','46069','CLINICA ORTOPEDICA PAULISTA LTDA','03140417000148',...,'1.0','400.14','0000282693','0000282693','2016-01-02','201601',NaN,NaN,NaN,NaN
1,'Seguradora Especializada em Saude','48','BRADESCO SAÃDE','42704626','1135485394','3280780','486879192','46069','CLINICA ORTOPEDICA PAULISTA LTDA','03140417000148',...,'1.0','400.14','0000282693','0000282693','2016-01-02','201601',NaN,NaN,NaN,NaN
2,'Seguradora Especializada em Saude','48','BRADESCO SAÃDE','42704626','1135485394','3280780','486879192','46069','CLINICA ORTOPEDICA PAULISTA LTDA','03140417000148',...,'1.0','400.14','0000282693','0000282693','2016-01-02','201601',NaN,NaN,NaN,NaN
3,'Seguradora Especializada em Saude','48','BRADESCO SAÃDE','42704626','1135485394','3280780','486879191','46069','CLINICA ORTOPEDICA PAULISTA LTDA','03140417000148',...,'1.0','400.14','0000282693','0000282693','2016-01-02','201601',NaN,NaN,NaN,NaN
4,'Seguradora Especializada em Saude','48','BRADESCO SAÃDE','42704626','1135485394','3280780','486879191','46069','CLINICA ORTOPEDICA PAULISTA LTDA','03140417000148',...,'1.0','400.14','0000282693','0000282693','2016-01-02','201601',NaN,NaN,NaN,NaN


In [52]:
df['\'data\''].loc[df['\'servico\''].str.contains(r'^.{4}(?!\d+).*')] = 'Deslocado'
df['\'tipo_item\''].loc[df['\'data\''] == 'Deslocado'] = df['\'desc_tipoalta\''].loc[df['\'data\''] == 'Deslocado']
df['\'servico\''].loc[df['\'data\''] == 'Deslocado'] = df['\'descricao_despesa\''].loc[df['\'data\''] == 'Deslocado']
df['\'descricao_despesa\''].loc[df['\'data\''] == 'Deslocado'] = df['\'qtde\''].loc[df['\'data\''] == 'Deslocado']
df['\'qtde\''].loc[df['\'data\''] == 'Deslocado'] = df['\'valor\''].loc[df['\'data\''] == 'Deslocado']
df['\'valor\''].loc[df['\'data\''] == 'Deslocado'] = df['\'origem\''].loc[df['\'data\''] == 'Deslocado']

Selecionando apenas os campos relevantes para esta análise:

In [53]:
selected_df = pd.concat([df["'id_conta'"], df["'servico'"], df["'descricao_despesa'"], df["'tipo_item'"]], axis=1)

Removendo as aspas das labels para facilitar o trabalho:

In [54]:
selected_df.columns = ["id_conta", "servico", "descricao", "tipo_item"]

Vamos trabalhar apenas com os id_conta atrelados à mais de um serviço:

In [55]:
count_df = selected_df.groupby("id_conta").count()

In [56]:
def get_count(acc_id):
    return count_df.lookup([acc_id], ["servico"])[0]

In [57]:
selected_df["count"] = selected_df["id_conta"].map(get_count)

In [58]:
selected_df = selected_df[selected_df["count"] > 1].drop("count", axis=1)

Algumas observações possuem apenas " no campo serviço, vamos filtra-las também:

In [59]:
selected_df = selected_df[selected_df["servico"] != '"']
selected_df = selected_df[selected_df["servico"] != "''"]

In [60]:
selected_df.head()

,id_conta,servico,descricao,tipo_item
19,'1135853444','30726042','Artrotomia - tratamento cirÃºrgico','PROCEDIMENTO'
20,'1135853444','30726239','Realinhamentos do aparelho extensor - tratame...,'PROCEDIMENTO'
21,'1135853444','30726034','Artroplastia total de joelho com implantes - ...,'PROCEDIMENTO'
22,'1135853446','30726042','Artrotomia - tratamento cirÃºrgico','PROCEDIMENTO'
23,'1135853446','30726239','Realinhamentos do aparelho extensor - tratame...,'PROCEDIMENTO'


O resultado que estamos buscando é uma tabela que mostre, para cada serviço, os serviços que aparecem com mais frequencia junto dele. Para isso, começamos definindo uma função para selecionar apenas os id_conta que contem um dado serviço:

In [61]:
def select_ids_containing_service(service):
    acc_ids = selected_df[selected_df["servico"] == service]["id_conta"].values
    return selected_df.loc[selected_df["id_conta"].isin(acc_ids)]

Agora, definimos uma função para computar as frequencias de cada serviço dentro das dataframes selecionadas pela função acima:

In [62]:
def get_percentage(service, service_df):
    unique_acc_ids = len(service_df["id_conta"].unique())
    count_num = len(service_df[service_df["servico"] == service]["id_conta"].unique())
    return (float(count_num) / unique_acc_ids) * 100.0

E computamos a frequencia relativa a cada serviço principal para cada serviço atrelado. Apenas serviços que constam em mais de 1 id_conta são considerados serviços principais. 

In [142]:
rel_freq_dict = {}
for service in selected_df["servico"].unique():
    rel_freq_dict[service] = []
    service_df = select_ids_containing_service(service)
    if len(service_df["id_conta"].unique()) == 1: # only working with services in more than a single id
        continue
    for related_service in service_df["servico"].unique():
        if related_service == service or related_service == master: # relation to itself and master is always 100%
            continue 
        relative_freq = get_percentage(related_service, service_df)
        rel_freq_dict[service].append((related_service, relative_freq))

E montamos a tabela para exibir nossos resultados:

In [143]:
index = []
for main_service in rel_freq_dict.keys():
    for related_service, freq in rel_freq_dict[main_service]:
        index.append((main_service, related_service))

In [144]:
mul_index = pd.MultiIndex.from_tuples(index, names=('Servico', 'Servicos Correlatos'))

In [145]:
corr_df = pd.DataFrame([freq for service in rel_freq_dict.keys() for (rel_serv, freq) in rel_freq_dict[service]], mul_index, ["%"])

E, para cada serviço principal, ordenamos os serviços relacionados por frequencia relativa:

In [146]:
sorted_df = corr_df.sort_values('%', ascending=False).sort_index(level='Servico', sort_remaining=False)
sorted_df

%
Servico      Servicos Correlatos       
'0000001639' '70705330'           100.0
             '70176108'           100.0
             '40804054'           100.0
             '90047249'           100.0
             '10102019'           100.0
             '0000027808'         100.0
             '90094654'           100.0
             '90090730'           100.0
             '70360812'           100.0
             '0000128569'          50.0
             '0000154311'          50.0
             '70193622'            50.0
             '90137477'            50.0
             '90045980'            50.0
             '90099419'            50.0
             '70270082'            50.0
             '70139105'            50.0
             '80042414'            50.0
             '20104073'            50.0
             '0000060583'          50.0
             '80053343'            50.0
             '0000167323'          50.0
             '80021042'            50.0
             '80051510'            50.0
             '80052908'            50.0
             '70349690'            50.0
             '70139172'            50.0
             '0000048140'          50.0
             '90184378'            50.0
             '70090050'            50.0
...                                 ...
'99999999'   '90142063'            50.0
             '0000167323'          50.0
             '20104073'            50.0
             '40804054'            50.0
             '80021042'            50.0
             '70090050'            50.0
             '80041043'            50.0
             '70777730'            50.0
             '90018591'            50.0
             '0000027825'          50.0
             '0000044131'          50.0
             '70799172'            50.0
             '80011012'            50.0
             '80053920'            50.0
             '80053904'            50.0
             '0000048140'          50.0
             '80042414'            50.0
             '0000154311'          50.0
             '0000128569'          50.0
             '0000027808'          50.0
             '70139105'            50.0
             '70270082'            50.0
             '90094654'            50.0
             '90099419'            50.0
             '90045980'            50.0
             '90137477'            50.0
             '70193622'            50.0
             '70139172'            50.0
             '90099370'            50.0
             '90184378'            50.0

[194376 rows x 1 columns]

Vamos agora preparar visualizações mais claras dos resultados. Primeiro, uma função para pegar a descrição de cada serviço para facilitar a compreensão: 

In [147]:
def get_description(service):
    return selected_df[selected_df["servico"] == service]["descricao"].iloc[0]

E agora selecionamos os 6 serviços que aparecem mais frequentemente junto ao master:

In [150]:
top_services = list(sorted_df.loc[master].head(6).index) # first item is the master itself

E vemos quais foram os serviços que mais apareceram junto a cada um deles:

In [151]:
for service in top_services:
    top_service_df = sorted_df.loc[service].head(5)
    top_service_df["Descricao"] = [get_description(item) for item in top_service_df.index]
    print "--------- " + service + ": " + get_description(service) +  " ---------" 
    display(top_service_df)
    print ""

--------- '40304361': 'HEMOGRAMA COM CONTAGEM DE PLAQUETAS OU FRAES ERITR' ---------


,%,Descricao
Servicos Correlatos,,
'40301630',100.000000,'CREATININA'
'40302580',100.000000,'URIA'
'40302318',95.454545,'POTSSIO'
'40302423',95.454545,'SDIO'
'40302237',81.818182,'MAGNSIO'



--------- '40302580': 'URIA' ---------


,%,Descricao
Servicos Correlatos,,
'40304361',100.000000,'HEMOGRAMA COM CONTAGEM DE PLAQUETAS OU FRAES ...
'40301630',100.000000,'CREATININA'
'40302318',95.454545,'POTSSIO'
'40302423',95.454545,'SDIO'
'40304590',81.818182,'Tempo de protrombina - determinacao'



--------- '90196031': 'DIPIRONA 1G AMPOLA 2ML' ---------


...

,%,Descricao
Servicos Correlatos,,
'40302580',100.000000,'URIA'
'40304361',100.000000,'HEMOGRAMA COM CONTAGEM DE PLAQUETAS OU FRAES ...
'40302318',95.454545,'POTSSIO'
'40302423',95.454545,'SDIO'
'40304590',81.818182,'Tempo de protrombina - determinacao'



--------- '40302423': 'SDIO' ---------


,%,Descricao
Servicos Correlatos,,
'40301630',100.000000,'CREATININA'
'40304361',100.000000,'HEMOGRAMA COM CONTAGEM DE PLAQUETAS OU FRAES ...
'40302580',100.000000,'URIA'
'40302318',95.238095,'POTSSIO'
'40302237',80.952381,'MAGNSIO'



--------- '40302318': 'POTSSIO' ---------


,%,Descricao
Servicos Correlatos,,
'40301630',100.000000,'CREATININA'
'40304361',100.000000,'HEMOGRAMA COM CONTAGEM DE PLAQUETAS OU FRAES ...
'40302580',100.000000,'URIA'
'40302423',95.238095,'SDIO'
'40302237',80.952381,'MAGNSIO'
